In [1]:
import gym
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

/Users/mattfeng/envs/deep/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
class Agent(object):
    def __init__(self, lr, s_size, a_size, h_size):
        # FF part of the network: State -> Action
        self.state_in = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size, biases_initializer=None,
                                      activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden, a_size, biases_initializer=None,
                                           activation_fn=tf.nn.softmax)
        print("tf.shape(self.output) =", tf.shape(self.output))
        self.chosen_action = tf.argmax(self.output, axis=1)
        
        # Training
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        self.indices = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        print("indices shape:", tf.shape(self.indices))
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indices)
        
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) * self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for ix, var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32, name="{}_holder".format(str(ix)))
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss, tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

In [3]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
tf.reset_default_graph()

agent = Agent(1e-2, 4, 2, 8)
saver = tf.train.Saver()

sess = tf.Session()
saver.restore(sess, './cartpole/cartpole')

max_steps = 10000

s = env.reset()
for step in range(max_steps):
    a_dist = sess.run(agent.output, feed_dict={agent.state_in: [s]})
    a = np.random.choice(range(len(a_dist[0])), p=a_dist[0])
    
    s, r, done, _ = env.step(a)
    env.render()
    
    if done:
        break

tf.shape(self.output) = Tensor("Shape:0", shape=(2,), dtype=int32)
indices shape: Tensor("Shape_3:0", shape=(1,), dtype=int32)
INFO:tensorflow:Restoring parameters from ./cartpole/cartpole


/Users/mattfeng/envs/deep/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
